In [1]:
from fastapi import FastAPI, requests, HTTPException
import uvicorn
import os
from dotenv import load_dotenv
import psycopg2
import json
import pandas as pd

In [2]:
load_dotenv()
    
#Leer Las Variables:
host = os.getenv("DB_HOST_AWS")
username = os.getenv("DB_USER_AWS")
password = os.getenv("DB_PASSWORD_AWS")
database = os.getenv("DB_DATABASE_AWS")
port = os.getenv("DB_PORT_AWS")


In [4]:
try:
    connection = psycopg2.connect(host=host,
                                database=database,
                                user=username,
                                password=password,
                                port=port,
                                sslmode="require")
    
    print("Conexión exitosa a la base de datos PostgreSQL con SSL")
except psycopg2.OperationalError as e:
    raise HTTPException(status_code=400, detail="Error de conexión: " + str(e))
except Exception as error:
    raise HTTPException(status_code=400, detail="Error desconocido: " + str(e))



Conexión exitosa a la base de datos PostgreSQL con SSL


In [ ]:
# Escribe la consulta SQL
query = "SELECT * FROM preguntas_front"

In [12]:
# Usa pandas para ejecutar la consulta y convertirla en un DataFrame
df = pd.read_sql_query(query, connection)
df.head(50)

C:\Users\2012f\AppData\Local\Temp\ipykernel_3532\1699526117.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,id_categoria,titulo_categoria,texto_pregunta,texto_opcion
0,1.1.1,Tengo VIH,¿Cuándo te diagnosticaron?,Hace menos de 6 meses
1,1.1.2,Tengo VIH,¿Cuándo te diagnosticaron?,Entre 6 meses y un año
2,1.1.3,Tengo VIH,¿Cuándo te diagnosticaron?,Hace más de un año
3,1.1.4,Tengo VIH,¿Estás en tratamiento TAR?,Sí
4,1.1.5,Tengo VIH,¿Estás en tratamiento TAR?,No
5,1.1.6,Tengo VIH,¿Estás en tratamiento TAR?,No estoy seguro
6,1.1.7,Tengo VIH,¿Tienes acceso a un médico?,Sí
7,1.1.8,Tengo VIH,¿Tienes acceso a un médico?,No
8,1.2.1,Creo que me he expuesto al virus,¿Cuándo ocurrió la posible infección?,Últimas 72h
9,1.2.2,Creo que me he expuesto al virus,¿Cuándo ocurrió la posible infección?,Hace más de 72h


In [ ]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)

{
    "1.1.1": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace menos de 6 meses"
            ]
        }
    },
    "1.1.2": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Entre 6 meses y un año"
            ]
        }
    },
    "1.1.3": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace más de un año"
            ]
        }
    },
    "1.1.4": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
                "Sí"
            ]
        }
    },
    "1.1.5": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
                "No"
            ]
        }
    },
    "1.1.6": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
     

In [11]:
df = pd.read_csv("ddbb_prueb.csv")
df

,id_categoria,titulo_categoria,texto_pregunta,texto_opcion
0,1.1,Tengo VIH,¿Cuándo te diagnosticaron?,Hace menos de 6 meses
1,1.1,Tengo VIH,¿Cuándo te diagnosticaron?,Entre 6 meses y un año
2,1.1,Tengo VIH,¿Cuándo te diagnosticaron?,Hace más de un año
3,1.1,Tengo VIH,¿Estás en tratamiento TAR?,Sí
4,1.1,Tengo VIH,¿Estás en tratamiento TAR?,No
5,1.1,Tengo VIH,¿Estás en tratamiento TAR?,No estoy seguro
6,1.1,Tengo VIH,¿Tienes acceso a un médico?,Sí
7,1.1,Tengo VIH,¿Tienes acceso a un médico?,No
8,1.2,Creo que me he expuesto al virus,¿Cuándo ocurrió la posible infección?,Últimas 72h
9,1.2,Creo que me he expuesto al virus,¿Cuándo ocurrió la posible infección?,Hace más de 72h


In [12]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)


{
    "1.1": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace menos de 6 meses",
                "Entre 6 meses y un año",
                "Hace más de un año"
            ],
            "¿Estás en tratamiento TAR?": [
                "Sí",
                "No",
                "No estoy seguro"
            ],
            "¿Tienes acceso a un médico?": [
                "Sí",
                "No"
            ]
        }
    },
    "1.2": {
        "titulo": "Creo que me he expuesto al virus",
        "preguntas": {
            "¿Cuándo ocurrió la posible infección?": [
                "Últimas 72h",
                "Hace más de 72h"
            ]
        }
    },
    "2.1": {
        "titulo": "Personal sanitario",
        "preguntas": {
            "¿que necesitas?": [
                "Manejo clínico de pacientes con VIH",
                "Protocolo PEP"
            ]
        }
    },
    "2.2": {
        "titulo